### Imports

In [ ]:
import logging

from IPython.core.display import display, HTML

from scripts.main import *
from scripts.retrieve_data import *
from scripts.extract_data import *

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

# set the width of the notebook
display(HTML("<style>.container { width:95% !important; }</style>"))

### Prepare for tests

In [ ]:
create_logger()
config = load_config()

In [ ]:
df_studies = find_studies_for_day(config, config['main']['start_date'].replace('-', ''), ['PT', 'NM'])
df_series = find_series_for_studies(config, df_studies)

### Test single association

In [ ]:
image_level_filters = ['Patient ID', 'Series Date', 'Series Instance UID', 'Modality']
to_fetch_fields = ['InstanceNumber', 'ManufacturerModelName', 'AcquisitionTime', 'Modality',
                   'ImageType', 'ActualFrameDuration', 'NumberOfFrames', '0x00540032', '0x00540052']
# create the query dataset
series_row = df_series.iloc[1, :]
query_ds = create_dataset_from_dataframe_row(series_row, 'IMAGE', incl=image_level_filters, add_instance_number_filter=True)
# add some more filters for the 'PT' modality
if series_row['Number of Series Related Instances'] != '1': image_number_filter = ['1', series_row['Number of Series Related Instances']]
else: image_number_filter = '1'
# add the instance number filters for the 'PT' & 'CT' modalities
if series_row['Modality'] == 'PT' or series_row['Modality'] == 'CT': query_ds.InstanceNumber = image_number_filter
# display the Dataset
logging.info('Query Dataset:')
for s in str(query_ds).split('\n'): logging.info('    ' + s)

df, datasets = get_data_single(config, query_ds, to_fetch_fields)
df

### Test multiple association

In [ ]:
# fields to use as filters to get the image
image_level_filters = ['Patient ID', 'Series Date', 'Series Instance UID', 'Modality']
# fields to fetch from the DICOM header
to_fetch_fields = ['InstanceNumber', 'ManufacturerModelName', 'AcquisitionTime', 'Modality',
                   'ImageType', 'ActualFrameDuration', 'NumberOfFrames', '0x00540032', '0x00540052']

In [ ]:
# create the query datasets for each row
query_datasets = df_series.apply(
    lambda row:
    create_dataset_from_dataframe_row(row, 'IMAGE', incl=image_level_filters),
    axis=1)

In [ ]:
# fetch the data (C-MOVE)
df_info, _ = get_data(config, query_datasets, to_fetch_fields)